# 질병 BBOX

BBOX_List_2017.csv에 제공된 질병 정답 데이터 시각화 자료

In [3]:
import os
import cv2
import pandas as pd
import glob
from tqdm import tqdm

def visualize_bboxes_from_csv(img_dir, csv_path, output_dir):
    # 1. 출력 폴더 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 출력 폴더 생성 완료: {output_dir}")

    # 2. CSV 파일 로드
    print(f"📄 CSV 파일 로드 중: {csv_path}")
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"❌ CSV 파일을 찾을 수 없습니다: {csv_path}")
        return

    # 3. 이미지 파일 리스트 확보
    # 폴더 내의 png, jpg 파일 등을 모두 찾습니다.
    img_extensions = ['*.png', '*.jpg', '*.jpeg']
    img_files = []
    for ext in img_extensions:
        img_files.extend(glob.glob(os.path.join(img_dir, ext)))
    
    # 빠른 검색을 위해 파일명 -> 전체 경로 매핑 딕셔너리 생성
    img_path_map = {os.path.basename(f): f for f in img_files}
    
    print(f"🚀 총 {len(img_files)}장의 이미지와 CSV 데이터를 매칭하여 처리를 시작합니다...")

    # 4. 이미지별로 그룹화하여 처리 (한 이미지에 BBox가 여러 개일 수 있음)
    grouped = df.groupby('Image Index')

    processed_count = 0

    for filename, group in tqdm(grouped):
        # 해당 이미지가 실제 폴더에 있는지 확인
        if filename not in img_path_map:
            continue # 폴더에 없는 이미지는 스킵

        full_img_path = img_path_map[filename]
        
        # 이미지 로드 (컬러로 로드해야 빨간 박스 표시 가능)
        img = cv2.imread(full_img_path)
        if img is None:
            continue

        # 해당 이미지에 대한 모든 BBox 그리기
        for _, row in group.iterrows():
            try:
                # CSV 컬럼명에 맞춰 데이터 추출
                label = row['Finding Label']
                x = int(row['Bbox [x'])
                y = int(row['y'])
                w = int(row['w'])
                h = int(row['h]'])
                
                # BBox 그리기 (빨간색, 두께 2)
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
                
                # 텍스트 라벨 (질병명) 추가
                # 텍스트 배경(검은색) 추가하여 가독성 높임
                text = label
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.6
                thickness = 1
                
                (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
                
                # 텍스트 배경 박스
                cv2.rectangle(img, (x, y - text_height - 5), (x + text_width, y), (0, 0, 255), -1)
                
                # 텍스트 쓰기 (흰색)
                cv2.putText(img, text, (x, y - 5), font, font_scale, (255, 255, 255), thickness)
                
            except Exception as e:
                print(f"⚠️ 데이터 파싱 오류 ({filename}): {e}")
                continue

        # 결과 저장
        save_path = os.path.join(output_dir, filename)
        cv2.imwrite(save_path, img)
        processed_count += 1

    print(f"\n✅ 처리 완료: 총 {processed_count}장의 이미지가 저장되었습니다.")
    print(f"📁 저장 위치: {output_dir}")

# --- 실행 설정 ---
if __name__ == "__main__":
    # 사용자 경로 설정
    INPUT_IMG_DIR = r"D:\lung_xray\final_imgsets"
    CSV_FILE_PATH = r"D:\lung_xray\BBox_List_2017.csv"
    OUTPUT_BBOX_DIR = r"D:\lung_xray\disease_BBOX_finalset"

    visualize_bboxes_from_csv(INPUT_IMG_DIR, CSV_FILE_PATH, OUTPUT_BBOX_DIR)

📁 출력 폴더 생성 완료: D:\lung_xray\disease_BBOX_finalset
📄 CSV 파일 로드 중: D:\lung_xray\BBox_List_2017.csv
🚀 총 52208장의 이미지와 CSV 데이터를 매칭하여 처리를 시작합니다...


100%|██████████| 880/880 [00:18<00:00, 48.74it/s]


✅ 처리 완료: 총 217장의 이미지가 저장되었습니다.
📁 저장 위치: D:\lung_xray\disease_BBOX_finalset
